In [1]:
import pandas as pd
import json
import numpy as np

# Lendo arquivo de dados
Nesse primeiro bloco vamos ler as dados salvos no arquivo de log e carregá-los em um dataframe. 

In [2]:
with open('twitter.log', 'r') as f:
    data = f.readlines()
data = [ json.loads(line) for line in data]


In [3]:
data_f = pd.DataFrame(data) 
# total de msgs
len(data_f) 

23355

Nessa parte da análise vamos verificar quantas menssagens tem a geo localização habilitada. 

In [4]:
enable_geo = data_f[data_f['coordinates'].notnull()]
len(enable_geo) 

1158

Verificando as fontes das tweets


In [5]:
data_f['count_groupby'] = 1
data_f[['source', 'count_groupby']] \
         .groupby('source').count()  

,count_groupby
source,
"<a href=""http://foursquare.com"" rel=""nofollow"">Foursquare</a>",138
"<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",805
"<a href=""http://laconversa.com"" rel=""nofollow"">Tendências São Paulo</a>",1
"<a href=""http://tapbots.com/software/tweetbot/mac"" rel=""nofollow"">Tweetbot for Mac</a>",3
"<a href=""http://tapbots.com/tweetbot"" rel=""nofollow"">Tweetbot for iΟS</a>",92
"<a href=""http://trendsmap.com/"" rel=""nofollow"">Trendsmap Alerting</a>",12
"<a href=""http://tweetmyjobs.com"" rel=""nofollow"">SafeTweet by TweetMyJOBS</a>",1
"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",4286
"<a href=""http://twitter.com/#!/download/ipad"" rel=""nofollow"">Twitter for iPad</a>",32


Quais fontes tem mais re-tweets? 

In [6]:
retweets = data_f[['retweeted', 'count_groupby']] .groupby('retweeted').count()      
    
retweets.head() 

,count_groupby
retweeted,
False,23355


Explorando os dados dos usuarios

In [7]:
users = data_f['user'].apply(pd.Series)      



In [8]:
#users['verified'].groupby('verified').count()     

users['count_groupby'] = 1
user_verif = users.groupby('verified').count()['count_groupby'] 
user_verif.head() 

verified
False    23287
True        68
Name: count_groupby, dtype: int64

In [52]:
data = pd.concat([data_f, users], axis=1)     
data.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,url,utc_offset,verified,count_groupby
0,None,"{'type': 'Point', 'coordinates': [-46.57978356...",Mon Aug 07 00:50:03 +0000 2017,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0,False,low,...,000000,False,False,enrico1830,9776,Brasilia,http://Instagram.com/enrico1830,-10800.0,False,1
1,None,None,Mon Aug 07 00:50:04 +0000 2017,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,000000,False,False,gabwhenicome,37702,Pacific Time (US & Canada),https://www.instagram.com/thecure.s/,-25200.0,False,1
2,None,None,Mon Aug 07 00:50:04 +0000 2017,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,333333,False,False,Lauralemec,11898,Brasilia,None,-10800.0,False,1
3,None,None,Mon Aug 07 00:50:05 +0000 2017,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,333333,True,False,Gordin53,1403,None,None,NaN,False,1
4,None,None,Mon Aug 07 00:50:05 +0000 2017,"[0, 15]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 894359957759569920, 'id_str'...",NaN,0,False,low,...,333333,True,False,aoutrafernanda,75102,Santiago,None,-14400.0,False,1


Quais variáveis do usuários são mais relevantes  para ter mais seguidores? 

In [55]:
data.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted', 'source', 'text',
       'timestamp_ms', 'truncated', 'user', 'count_groupby',
       'contributors_enabled', 'created_at', 'default_profile',
       'default_profile_image', 'description', 'favourites_count',
       'follow_request_sent', 'followers_count', 'following', 'friends_count',
       'geo_enabled', 'id', 'id_str', 'is_translator', 'lang', 'listed_count',
       'location', 'name', 'notifications', 'profile_background_color',
       'profile_background_image_url

In [10]:
top_users = users[users['followers_count'] > 100000]       


In [11]:
len(top_users.name.unique()) 

25

In [59]:
top_users.description.head(1000)

566      Narrador esportivo. Qual quer duvida compre o ...
572                                                   None
2344     Comentarista do #CBLOL (League of Legends Cast...
2970              viciado na loucura por pura malandragem.
3017     Contato Profissional: contatoflokiis@gmail.com...
3136     Como chegar nela eu não sei ela tão interessan...
3235     Como chegar nela eu não sei ela tão interessan...
3355     Prefeito eleito com mais de 3 milhões de votos...
3426     Espaço para críticas, sugestões e elogios sobr...
3476     Espaço para críticas, sugestões e elogios sobr...
3709     Sou um Zeus das 2 Rodas ! ❤. Snapchat: aln1001...
3910     🇧🇷🇮🇹 Brazilian/Italian | Marketing | Music Ind...
4461              viciado na loucura por pura malandragem.
4665     Narrador esportivo. Qual quer duvida compre o ...
5126                                                 a nao
5292     Narrador esportivo. Qual quer duvida compre o ...
6393     Espaço para críticas, sugestões e elogios sobr.

In [13]:
len(users.name.unique())

6430

In [14]:
text = data_f[['source', 'text']].where(data_f['source'].str.contains('twitter.com'))      

In [15]:
import re

texto = text['text'].dropna()

texto_limpo_sem_emoji = texto[~texto.str.contains(u"[^\U00000000-\U0000d7ff\U0000e000-\U0000ffff]").dropna()]


In [16]:

files_number = 5
for i in range(5):
    chunk = texto_limpo_sem_emoji.sample(frac=0.25)
    chunk.to_csv('tweets_%s.csv'% (i)) 

In [17]:
teste = text.head(100)

In [18]:
len(texto)

22273

In [19]:
len(texto_limpo_sem_emoji)

19838

In [31]:
import nltk

In [21]:
texto.head()

1    se tu era trouxa tu continua sendo\nsó q pior ...
2    sera q alguem pode pular pra parte q eu to apo...
3    @gleisi  Vc nem vcs vão conseguir voltar ao po...
4              vai começar GOT https://t.co/NU6NROXES2
5                           Minha mãe tá loucaaaaaaaaa
Name: text, dtype: object

In [47]:

from nltk import word_tokenize, pos_tag, ne_chunk
 
sentence = "O Chico Ferramenta trabalha em Ipatinga"
 
print (ne_chunk(pos_tag(word_tokenize(sentence))))
'''
def extract_entity_names(t):
    entity_names = []
    
    if hasattr(t, 'node') and t.node:
        if t.node == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))
                
    return entity_names

entity_names = []
for tree in chunked_sentences:
    # Print results per sentence
    # print extract_entity_names(tree)
    
    entity_names.extend(extract_entity_names(tree))

# Print all entity names
#print entity_names

# Print unique entity names
'''
#print(chunked_sentences)

(S
  O/NNP
  Chico/NNP
  Ferramenta/NNP
  trabalha/NN
  em/NN
  (PERSON Ipatinga/NNP))


"\ndef extract_entity_names(t):\n    entity_names = []\n    \n    if hasattr(t, 'node') and t.node:\n        if t.node == 'NE':\n            entity_names.append(' '.join([child[0] for child in t]))\n        else:\n            for child in t:\n                entity_names.extend(extract_entity_names(child))\n                \n    return entity_names\n\nentity_names = []\nfor tree in chunked_sentences:\n    # Print results per sentence\n    # print extract_entity_names(tree)\n    \n    entity_names.extend(extract_entity_names(tree))\n\n# Print all entity names\n#print entity_names\n\n# Print unique entity names\n"

In [24]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True